In [1]:
import os

In [2]:
pwd 

'c:\\Users\\pmore4\\Desktop\\vs code\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'c:\\Users\\pmore4\\Desktop\\vs code\\Text-Summarizer-Project'

In [28]:
!pip install sentencepiece


   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ---------- ----------------------------- 262.1/991.5 kB ? eta -:--:--
   --------------------- ------------------ 524.3/991.5 kB 1.0 MB/s eta 0:00:01
   ------------------------------- -------- 786.4/991.5 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 991.5/991.5 kB 1.2 MB/s eta 0:00:00


In [5]:
from  dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir :Path
    data_path : Path
    tokenizer_name : Path

In [6]:
# define ConfigurationManager
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [12]:
!pip install datasets

  Using cached pyarrow-17.0.0-cp39-cp39-win_amd64.whl.metadata (3.4 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp39-cp39-win_amd64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp39-cp39-win_amd64.whl.metadata (4.3 kB)
  Using cached yarl-1.9.4-cp39-cp39-win_amd64.whl.metadata (32 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
   ---------------------------------------- 0.0/527.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.3 kB ? eta -:--:--
   ------------------- -------------------- 262.1/527.3 kB ? eta -:--:--
   ------------------- -------------------- 262.1/527.3 kB ? eta -:--:--
   -------------------------------------- 527.3/527.3 kB 736.4 kB/s eta 0:00:00
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached pyarrow-17.0.0-cp

In [8]:
# componenets

import os
from textSummarizer.logging import logger
from tranasformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

c:\Users\pmore4\Desktop\vs code\Text-Summarizer-Project\textsummarize\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Program Files\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\Python\Python39\lib\runpy.py", line 87, in _run_code
    exec(code, run_gl

[2024-08-28 10:31:35,854: INFO: config: PyTorch version 2.4.0 available.]


In [11]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name,use_fast=False)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [12]:

try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()

except Exception as e:
    raise e

[2024-08-28 10:32:51,873: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-28 10:32:51,877: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-28 10:32:51,880: INFO: common: Created directory at path :artifacts]
[2024-08-28 10:32:51,882: INFO: common: Created directory at path :artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\pmore4\Desktop\vs code\Text-Summarizer-Project\textsummarize\lib\site-packages\transformers\tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 48677.56 examples/s]
